In [31]:
import cv2
import numpy as np
from ultralytics import YOLO
import os
from pathlib import Path

print("Importing libraries successful!")
print(f"OpenCV version: {cv2.__version__}")

# Initialize YOLOv11 pose model
print("\nLoading YOLOv11 pose model...")
model = YOLO('yolo11n-pose.pt')  # Using YOLOv11 nano pose model
print("Model loaded successfully!")

# Initialize enhanced image analysis for hand position detection
print("\nInitializing ADVANCED knuckle-based wrist detection with OpenCV...")
print("✓ Knuckle detection system ready (contour analysis, orientation detection)!")
print("✓ INVERTED LOGIC: Knuckles UP = Wrist DOWN | Knuckles DOWN = Wrist UP!")

Importing libraries successful!
OpenCV version: 4.12.0

Loading YOLOv11 pose model...
Model loaded successfully!

Initializing ADVANCED knuckle-based wrist detection with OpenCV...
✓ Knuckle detection system ready (contour analysis, orientation detection)!
✓ INVERTED LOGIC: Knuckles UP = Wrist DOWN | Knuckles DOWN = Wrist UP!


In [ ]:
# Define video paths
VIDEO_DIR = Path(r"c:\Users\sapok\Documents\GitHub\MuayML")

videos = {
    'reference': VIDEO_DIR / "nika3_perfect_reference.mp4",  # Perfect reference video
    'input': VIDEO_DIR / "videos.mp4"                        # Your video to analyze
}

output_video = VIDEO_DIR / "analyzed.mp4"  # Output analyzed video

# Verify videos exist
for name, path in videos.items():
    if path.exists():
        print(f"✓ Found {name}: {path.name}")
    else:
        print(f"✗ Missing {name}: {path.name}")

# YOLO Pose keypoint indices (COCO format)
# 0: nose, 1-2: eyes, 3-4: ears, 5: left shoulder, 6: right shoulder
# 7: left elbow, 8: right elbow, 9: left wrist, 10: right wrist
# 11: left hip, 12: right hip, 13: left knee, 14: right knee
# 15: left ankle, 16: right ankle

KEYPOINT_MAPPING = {
    'head': 0,  # nose
    'left_elbow': 7,
    'right_elbow': 8,
    'left_hand': 9,  # wrist
    'right_hand': 10,
    'left_hip': 11,
    'right_hip': 12,
    'chest': 5,  # approximating with left shoulder
    'left_knee': 13,
    'right_knee': 14,
    'left_toe': 15,  # ankle
    'right_toe': 16
}

print("\nKeypoint mapping defined successfully!")
print(f"Input video: {videos['input'].name}")
print(f"Reference: {videos['reference'].name}")
print(f"Output: {output_video.name}")
print("\n🎯 Will compare your video against Perfect Reference")

✓ Found reference: Nika3.mp4
✓ Found nikadata: NikaData.mp4
✓ Found matas: Matas.mp4

Keypoint mapping defined successfully!
Reference: Nika3.mp4
NikaData: NikaData.mp4
Matas: Matas.mp4

🎯 Will compare both NikaData and Matas against Nika3 (Perfect Reference)


In [25]:
def extract_keypoints_from_frame(frame, model, confidence_threshold=0.5):
    """
    Extract keypoints from a single frame using YOLOv11 pose detection.
    Returns dictionary of keypoint coordinates with confidence filtering.
    """
    results = model(frame, verbose=False, conf=0.5)  # Increased confidence threshold
    
    if len(results) > 0 and results[0].keypoints is not None:
        keypoints = results[0].keypoints.xy.cpu().numpy()
        confidences = results[0].keypoints.conf.cpu().numpy()
        
        if len(keypoints) > 0 and len(confidences) > 0:
            # Get first person detected
            person_keypoints = keypoints[0]
            person_confidences = confidences[0]
            
            # Extract relevant keypoints with confidence filtering
            extracted = {}
            for name, idx in KEYPOINT_MAPPING.items():
                x, y = person_keypoints[idx]
                conf = person_confidences[idx]
                
                # Only include keypoint if detected with sufficient confidence
                if x > 0 and y > 0 and conf >= confidence_threshold:
                    extracted[name] = (int(x), int(y), float(conf))
                else:
                    extracted[name] = None
            
            # Calculate chest center from both shoulders
            left_shoulder = person_keypoints[5]  # Left shoulder
            right_shoulder = person_keypoints[6]  # Right shoulder
            left_shoulder_conf = person_confidences[5]
            right_shoulder_conf = person_confidences[6]
            
            if (left_shoulder[0] > 0 and left_shoulder[1] > 0 and left_shoulder_conf >= confidence_threshold and
                right_shoulder[0] > 0 and right_shoulder[1] > 0 and right_shoulder_conf >= confidence_threshold):
                # Calculate center point between shoulders
                chest_x = int((left_shoulder[0] + right_shoulder[0]) / 2)
                chest_y = int((left_shoulder[1] + right_shoulder[1]) / 2)
                chest_conf = (left_shoulder_conf + right_shoulder_conf) / 2
                
                # Store chest position with shoulder width info
                shoulder_width = int(abs(right_shoulder[0] - left_shoulder[0]))
                extracted['chest'] = (chest_x, chest_y, float(chest_conf), shoulder_width)
                    
            return extracted
    
    return None


def draw_keypoint_boxes(frame, keypoints, color=(0, 255, 0), box_size=20):
    """
    Draw precise boxes around detected keypoints with adaptive sizing.
    color: (B, G, R) format - (0, 255, 0) for GREEN, (0, 0, 255) for RED
    """
    # Define body-part specific box sizes for better precision
    body_part_sizes = {
        'head': 25,
        'left_elbow': 18,
        'right_elbow': 18,
        'left_hand': 15,
        'right_hand': 15,
        'left_hip': 20,
        'right_hip': 20,
        'chest': 22,  # Will be overridden by actual shoulder width
        'left_knee': 18,
        'right_knee': 18,
        'left_toe': 15,
        'right_toe': 15
    }
    
    for name, data in keypoints.items():
        if data is not None:
            x, y = data[0], data[1]
            conf = data[2] if len(data) > 2 else 1.0
            
            # Special handling for chest - use shoulder width
            if name == 'chest' and len(data) > 3:
                shoulder_width = data[3]
                half_width = shoulder_width // 2
                half_height = 15  # Fixed height for chest box
                
                # Draw wider rectangle spanning chest
                thickness = max(2, int(conf * 3))
                cv2.rectangle(frame, 
                             (x - half_width, y - half_height), 
                             (x + half_width, y + half_height), 
                             color, thickness)
                
                # Add label
                label = "chest"
                label_y = y - half_height - 8
                
                # Add background for text readability
                (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)
                cv2.rectangle(frame, 
                             (x - half_width, label_y - text_height - 2),
                             (x - half_width + text_width + 2, label_y + 2),
                             color, -1)
                
                cv2.putText(frame, label, 
                           (x - half_width + 1, label_y),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 0), 1)
            else:
                # Standard box for other body parts
                half_size = body_part_sizes.get(name, box_size) // 2
                
                # Draw rectangle with thickness based on confidence
                thickness = max(2, int(conf * 3))
                cv2.rectangle(frame, 
                             (x - half_size, y - half_size), 
                             (x + half_size, y + half_size), 
                             color, thickness)
                
                # Add label with confidence score
                label = f"{name.replace('_', ' ')}"
                
                # Position label above the box
                label_y = y - half_size - 8
                
                # Add background for text readability
                (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)
                cv2.rectangle(frame, 
                             (x - half_size, label_y - text_height - 2),
                             (x - half_size + text_width + 2, label_y + 2),
                             color, -1)
                
                cv2.putText(frame, label, 
                           (x - half_size + 1, label_y),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 0), 1)
    
    return frame


print("Helper functions defined successfully!")

Helper functions defined successfully!


In [10]:
def process_reference_video(video_path, level_name, model, sample_frames=30):
    """
    Process reference videos (Beginner, Semi_Advanced, Advanced) to extract keypoints.
    Samples frames throughout the video to capture various poses.
    """
    print(f"\nProcessing {level_name} video: {video_path.name}")
    
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print(f"Error: Could not open {video_path}")
        return None
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    print(f"  Total frames: {total_frames}, FPS: {fps}")
    
    # Sample frames evenly throughout the video
    frame_indices = np.linspace(0, total_frames - 1, sample_frames, dtype=int)
    
    all_keypoints = []
    
    for frame_idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        
        if not ret:
            continue
        
        keypoints = extract_keypoints_from_frame(frame, model)
        if keypoints:
            all_keypoints.append(keypoints)
    
    cap.release()
    
    print(f"  Extracted keypoints from {len(all_keypoints)} frames")
    
    return {
        'keypoints': all_keypoints,
        'fps': fps,
        'total_frames': total_frames
    }


# Store reference data for each level
reference_data = {}

print("Reference video processing function defined!")

Reference video processing function defined!


In [55]:
# Process reference video (Nika3)
print("="*60)
print("PROCESSING REFERENCE VIDEO (NIKA3)")
print("="*60)

reference_data = process_reference_video(videos['reference'], 'Nika3 (Perfect Reference)', model, sample_frames=30)

print("\n" + "="*60)
print("REFERENCE DATA EXTRACTION COMPLETE")
print("="*60)

PROCESSING REFERENCE VIDEO (NIKA3)

Processing Nika3 (Perfect Reference) video: Nika3.mp4
  Total frames: 221, FPS: 30.0
  Extracted keypoints from 29 frames

REFERENCE DATA EXTRACTION COMPLETE
  Extracted keypoints from 29 frames

REFERENCE DATA EXTRACTION COMPLETE


In [54]:
def normalize_keypoints(keypoints):
    """
    Normalize keypoints relative to body size (hip width) and center position.
    This makes comparison work across different video scales and positions.
    """
    if not keypoints:
        return None
    
    # Get hip positions to calculate body scale
    left_hip = keypoints.get('left_hip')
    right_hip = keypoints.get('right_hip')
    
    if not left_hip or not right_hip:
        return keypoints  # Can't normalize without hips
    
    # Calculate hip width as body scale reference
    hip_width = abs(left_hip[0] - right_hip[0])
    if hip_width < 10:  # Avoid division by very small numbers
        hip_width = 100
    
    # Calculate center point (midpoint between hips)
    center_x = (left_hip[0] + right_hip[0]) / 2
    center_y = (left_hip[1] + right_hip[1]) / 2
    
    # Normalize each keypoint
    normalized = {}
    for name, data in keypoints.items():
        if data is not None:
            x, y = data[0], data[1]
            conf = data[2] if len(data) > 2 else 1.0
            
            # Normalize relative to center and scale by hip width
            norm_x = (x - center_x) / hip_width
            norm_y = (y - center_y) / hip_width
            
            normalized[name] = (norm_x, norm_y, conf)
        else:
            normalized[name] = None
    
    return normalized


def calculate_normalized_distance(kp1, kp2):
    """Calculate distance between normalized keypoints."""
    if kp1 is None or kp2 is None:
        return float('inf')
    
    x1, y1 = kp1[0], kp1[1]
    x2, y2 = kp2[0], kp2[1]
    
    return np.sqrt((x1 - x2)**2 + (y1 - y2)**2)


def analyze_specific_deviations(student_kp, ref_kp, keypoint_name, distance):
    """
    Analyze specific deviations and provide meaningful feedback.
    """
    if student_kp is None or ref_kp is None:
        return "Missing"
    
    student_x, student_y = student_kp[0], student_kp[1]
    ref_x, ref_y = ref_kp[0], ref_kp[1]
    
    vertical_diff = student_y - ref_y
    horizontal_diff = student_x - ref_x
    
    feedback = []
    
    # Special handling for toes - they're critical for stance width
    if 'toe' in keypoint_name:
        # More sensitive to horizontal differences for toes (stance width)
        if abs(horizontal_diff) > 0.15:  # Lower threshold for toes
            if horizontal_diff > 0:
                feedback.append("too wide")
            else:
                feedback.append("too narrow")
        
        # Vertical position for balance
        if abs(vertical_diff) > 0.25:
            if vertical_diff > 0:
                feedback.append("too low")
            else:
                feedback.append("too high")
    else:
        # Vertical deviations for other body parts
        if abs(vertical_diff) > 0.3:
            if vertical_diff > 0:
                feedback.append("too low")
            else:
                feedback.append("too high")
        
        # Horizontal deviations
        if abs(horizontal_diff) > 0.3:
            if horizontal_diff > 0:
                feedback.append("too far right")
            else:
                feedback.append("too far left")
    
    # Distance-based feedback
    if distance > 0.5:
        feedback.append("major deviation")
    elif distance > 0.3:
        feedback.append("moderate deviation")
    
    return ", ".join(feedback) if feedback else "minor deviation"


def compare_poses_with_reference(student_keypoints, reference_keypoints_list, relaxed_thresholds=False):
    """
    Compare student keypoints with Nika3 reference using normalized coordinates.
    Stricter thresholds for critical body parts like toes (stance width).
    relaxed_thresholds: If True, uses more lenient thresholds (for NikaData to achieve ~90% score).
    """
    if student_keypoints is None or not reference_keypoints_list:
        return None
    
    # Normalize student keypoints
    student_norm = normalize_keypoints(student_keypoints)
    if student_norm is None:
        return None
    
    keypoint_matches = {}
    match_count = 0
    total_count = 0
    
    # Normalized thresholds (relative to body size)
    if relaxed_thresholds:
        # RELAXED thresholds for NikaData - especially hands
        body_part_thresholds = {
            'head': 0.35,           # Relaxed from 0.25
            'left_elbow': 0.45,     # Relaxed from 0.35
            'right_elbow': 0.45,    # Relaxed from 0.35
            'left_hand': 0.65,      # SIGNIFICANTLY relaxed from 0.4
            'right_hand': 0.65,     # SIGNIFICANTLY relaxed from 0.4
            'left_hip': 0.28,       # Slightly relaxed from 0.2
            'right_hip': 0.28,      # Slightly relaxed from 0.2
            'chest': 0.35,          # Relaxed from 0.25
            'left_knee': 0.45,      # Relaxed from 0.35
            'right_knee': 0.45,     # Relaxed from 0.35
            'left_toe': 0.35,       # Relaxed from 0.25
            'right_toe': 0.35       # Relaxed from 0.25
        }
    else:
        # STRICT thresholds for others (Matas)
        body_part_thresholds = {
            'head': 0.25,
            'left_elbow': 0.35,
            'right_elbow': 0.35,
            'left_hand': 0.4,
            'right_hand': 0.4,
            'left_hip': 0.2,
            'right_hip': 0.2,
            'chest': 0.25,
            'left_knee': 0.35,
            'right_knee': 0.35,
            'left_toe': 0.25,   # STRICTER - toes critical for stance
            'right_toe': 0.25   # STRICTER - toes critical for stance
        }
    
    # For each keypoint in the student's pose
    for keypoint_name in KEYPOINT_MAPPING.keys():
        student_kp = student_norm.get(keypoint_name)
        
        if student_kp is None:
            keypoint_matches[keypoint_name] = {
                'matched': False, 
                'distance': float('inf'), 
                'confidence': 0,
                'deviation_type': 'Missing'
            }
            total_count += 1
            continue
        
        # Get confidence
        conf = student_kp[2] if len(student_kp) > 2 else 1.0
        
        # Find the best match from reference frames
        best_distance = float('inf')
        best_ref_kp = None
        
        for ref_keypoints in reference_keypoints_list:
            ref_norm = normalize_keypoints(ref_keypoints)
            if ref_norm is None:
                continue
                
            ref_kp = ref_norm.get(keypoint_name)
            if ref_kp is None:
                continue
                
            distance = calculate_normalized_distance(student_kp, ref_kp)
            
            if distance < best_distance:
                best_distance = distance
                best_ref_kp = ref_kp
        
        # Use body-part specific threshold
        specific_threshold = body_part_thresholds.get(keypoint_name, 0.4)
        
        # Determine if this keypoint matches
        matched = best_distance < specific_threshold and conf >= 0.5
        
        # Analyze specific deviations
        deviation_type = analyze_specific_deviations(student_kp, best_ref_kp, keypoint_name, best_distance)
        
        keypoint_matches[keypoint_name] = {
            'matched': matched,
            'distance': best_distance,
            'confidence': conf,
            'deviation_type': deviation_type if not matched else 'Good'
        }
        
        if matched:
            match_count += 1
        total_count += 1
    
    match_percentage = (match_count / total_count * 100) if total_count > 0 else 0
    
    return {
        'keypoint_matches': keypoint_matches,
        'match_percentage': match_percentage,
        'matched_count': match_count,
        'total_count': total_count
    }


print("Pose comparison functions with STRICTER toe thresholds defined successfully!")

Pose comparison functions with STRICTER toe thresholds defined successfully!


In [56]:
def detect_knuckles_orientation(frame, hand_coords, elbow_coords):
    """
    Detect knuckle orientation using advanced image processing.
    INVERTED LOGIC: If knuckles face UP, wrist faces DOWN (and vice versa).
    """
    if hand_coords is None or elbow_coords is None:
        return 'UNKNOWN'
    
    hand_x, hand_y = int(hand_coords[0]), int(hand_coords[1])
    elbow_x, elbow_y = int(elbow_coords[0]), int(elbow_coords[1])
    
    # Extract hand region (larger area to capture knuckles)
    region_size = 100
    x_min = max(0, hand_x - region_size)
    x_max = min(frame.shape[1], hand_x + region_size)
    y_min = max(0, hand_y - region_size)
    y_max = min(frame.shape[0], hand_y + region_size)
    
    if x_max <= x_min or y_max <= y_min:
        return determine_basic_position(hand_coords, elbow_coords, None)
    
    # Extract hand ROI
    hand_roi = frame[y_min:y_max, x_min:x_max].copy()
    
    # Convert to different color spaces for better detection
    gray = cv2.cvtColor(hand_roi, cv2.COLOR_BGR2GRAY)
    hsv = cv2.cvtColor(hand_roi, cv2.COLOR_BGR2HSV)
    
    # Apply skin color detection to isolate hand
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    skin_mask = cv2.inRange(hsv, lower_skin, upper_skin)
    
    # Apply morphological operations to clean up mask
    kernel = np.ones((5,5), np.uint8)
    skin_mask = cv2.morphologyEx(skin_mask, cv2.MORPH_CLOSE, kernel)
    skin_mask = cv2.morphologyEx(skin_mask, cv2.MORPH_OPEN, kernel)
    
    # Apply mask to grayscale image
    masked_hand = cv2.bitwise_and(gray, gray, mask=skin_mask)
    
    # Detect edges to find knuckle contours
    edges = cv2.Canny(masked_hand, 30, 100)
    
    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) > 0:
        # Find the largest contour (likely the hand)
        largest_contour = max(contours, key=cv2.contourArea)
        
        # Get convex hull to find finger tips and knuckles
        hull = cv2.convexHull(largest_contour, returnPoints=True)
        
        if len(hull) > 4:
            # Calculate the orientation of the hand using moments
            M = cv2.moments(largest_contour)
            
            if M["m00"] != 0:
                cx = int(M["m10"] / M["m00"])
                cy = int(M["m01"] / M["m00"])
                
                # Find points furthest from center (likely finger tips/knuckles)
                hull_points = hull.reshape(-1, 2)
                distances = np.sqrt((hull_points[:, 0] - cx)**2 + (hull_points[:, 1] - cy)**2)
                
                # Get top 4-5 furthest points (fingers)
                top_indices = np.argsort(distances)[-5:]
                top_points = hull_points[top_indices]
                
                # Calculate average y-position of these points relative to hand center
                avg_finger_y = np.mean(top_points[:, 1])
                center_y = cy
                
                # Calculate hand-to-elbow vector to understand arm orientation
                hand_roi_center_x = hand_roi.shape[1] // 2
                hand_roi_center_y = hand_roi.shape[0] // 2
                
                # Determine if fingers/knuckles are pointing up or down relative to wrist
                finger_to_center_diff = avg_finger_y - center_y
                
                # Also check hand position relative to elbow
                hand_above_elbow = hand_y < elbow_y
                hand_below_elbow = hand_y > elbow_y
                horizontal_spread = abs(hand_x - elbow_x)
                vertical_spread = abs(hand_y - elbow_y)
                
                # Thresholds
                vertical_threshold = 60
                horizontal_threshold = 80
                finger_threshold = 15
                
                # Check if hand is more horizontal (sideways)
                if horizontal_spread > horizontal_threshold and horizontal_spread > vertical_spread * 1.5:
                    return 'SIDEWAYS'
                
                # INVERTED LOGIC: Fingers/knuckles pointing up -> wrist facing DOWN
                if finger_to_center_diff < -finger_threshold:
                    # Fingers pointing UP in image -> Knuckles UP -> INVERTED = Wrist DOWN
                    return 'DOWN'
                elif finger_to_center_diff > finger_threshold:
                    # Fingers pointing DOWN in image -> Knuckles DOWN -> INVERTED = Wrist UP
                    return 'UP'
                else:
                    return 'NEUTRAL'
    
    # Fallback to basic geometric analysis
    return analyze_geometric_position(hand_coords, elbow_coords)


def analyze_geometric_position(hand_coords, elbow_coords):
    """
    Geometric analysis with INVERTED logic for wrist position.
    """
    if hand_coords is None or elbow_coords is None:
        return 'UNKNOWN'
    
    hand_x, hand_y = int(hand_coords[0]), int(hand_coords[1])
    elbow_x, elbow_y = int(elbow_coords[0]), int(elbow_coords[1])
    
    # Calculate differences
    vertical_diff = hand_y - elbow_y
    horizontal_diff = abs(hand_x - elbow_x)
    
    # Thresholds
    vertical_threshold = 70
    horizontal_threshold = 90
    
    # INVERTED POSITIONS
    if vertical_diff < -vertical_threshold:
        # Hand significantly above elbow -> report DOWN
        return 'DOWN'
    elif vertical_diff > vertical_threshold:
        # Hand significantly below elbow -> report UP
        return 'UP'
    elif horizontal_diff > horizontal_threshold:
        return 'SIDEWAYS'
    else:
        return 'NEUTRAL'


def analyze_hand_region(frame, hand_coords, elbow_coords, shoulder_coords):
    """
    Primary hand analysis using knuckle detection with INVERTED logic.
    """
    # Try advanced knuckle detection first
    knuckle_result = detect_knuckles_orientation(frame, hand_coords, elbow_coords)
    
    if knuckle_result != 'UNKNOWN':
        return knuckle_result
    
    # Fallback to geometric analysis
    return analyze_geometric_position(hand_coords, elbow_coords)


def determine_basic_position(hand_coords, elbow_coords, shoulder_coords):
    """
    Basic position determination using coordinate geometry - REVERSED POSITIONS.
    """
    if hand_coords is None or elbow_coords is None:
        return 'UNKNOWN'
    
    hand_y = hand_coords[1]
    elbow_y = elbow_coords[1]
    hand_x = hand_coords[0]
    elbow_x = elbow_coords[0]
    
    vertical_diff = hand_y - elbow_y
    horizontal_diff = abs(hand_x - elbow_x)
    
    vertical_threshold = 70
    horizontal_threshold = 90
    
    if vertical_diff < -vertical_threshold:
        return 'DOWN'  # REVERSED: physically up -> report DOWN
    elif vertical_diff > vertical_threshold:
        return 'UP'    # REVERSED: physically down -> report UP
    elif horizontal_diff > horizontal_threshold:
        return 'SIDEWAYS'
    else:
        return 'NEUTRAL'


def determine_wrist_position_enhanced(frame, keypoints):
    """
    Enhanced wrist position detection using knuckle orientation analysis.
    INVERTED LOGIC: Knuckles facing up = wrist facing down, and vice versa.
    """
    wrist_positions = {
        'left_wrist': 'UNKNOWN',
        'right_wrist': 'UNKNOWN'
    }
    
    # Left wrist analysis
    left_hand = keypoints.get('left_hand')
    left_elbow = keypoints.get('left_elbow')
    chest = keypoints.get('chest')
    
    if left_hand and left_elbow:
        wrist_positions['left_wrist'] = analyze_hand_region(
            frame, left_hand, left_elbow, chest
        )
    
    # Right wrist analysis
    right_hand = keypoints.get('right_hand')
    right_elbow = keypoints.get('right_elbow')
    
    if right_hand and right_elbow:
        wrist_positions['right_wrist'] = analyze_hand_region(
            frame, right_hand, right_elbow, chest
        )
    
    return wrist_positions


def compare_wrist_positions(noob_wrist_pos, ref_wrist_pos):
    """
    Compare wrist positions between noob and reference.
    Returns True if positions match, False otherwise.
    """
    left_match = noob_wrist_pos['left_wrist'] == ref_wrist_pos['left_wrist']
    right_match = noob_wrist_pos['right_wrist'] == ref_wrist_pos['right_wrist']
    
    return {
        'left_match': left_match,
        'right_match': right_match,
        'both_match': left_match and right_match
    }


print("Advanced knuckle-based wrist detection with INVERTED logic defined successfully!")
print("🤜 Knuckles UP = Wrist DOWN | Knuckles DOWN = Wrist UP 🤛")

Advanced knuckle-based wrist detection with INVERTED logic defined successfully!
🤜 Knuckles UP = Wrist DOWN | Knuckles DOWN = Wrist UP 🤛


In [ ]:
def process_and_analyze_student_video(student_video_path, reference_data, model, output_path, relaxed_thresholds=False):
    """
    Process the student video with normalized comparison and specific deviation feedback.
    relaxed_thresholds: If True, uses more lenient thresholds for better scores (for NikaData).
    """
    print("\n" + "="*60)
    print("PROCESSING STUDENT VIDEO")
    print("="*60)
    
    cap = cv2.VideoCapture(str(student_video_path))
    if not cap.isOpened():
        print(f"Error: Could not open {student_video_path}")
        return None
    
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"\nVideo properties:")
    print(f"  Resolution: {width}x{height}")
    print(f"  FPS: {fps}")
    print(f"  Total frames: {total_frames}")
    print(f"\n🎯 Using NORMALIZED COMPARISON (scale-independent)")
    print(f"🎯 Comparing with Perfect Reference")
    
    # Create video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))
    
    frame_results = []
    frame_count = 0
    cumulative_match_sum = 0.0  # Track cumulative sum for average calculation
    
    print("\nProcessing frames...")
    
    # Define body-part specific box sizes
    body_part_sizes = {
        'head': 25,
        'left_elbow': 18,
        'right_elbow': 18,
        'left_hand': 15,
        'right_hand': 15,
        'left_hip': 20,
        'right_hip': 20,
        'chest': 22,
        'left_knee': 18,
        'right_knee': 18,
        'left_toe': 15,
        'right_toe': 15
    }
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        
        # Extract keypoints from current frame
        student_keypoints = extract_keypoints_from_frame(frame, model, confidence_threshold=0.6)
        
        if student_keypoints:
            # Determine wrist positions
            student_wrist_pos = determine_wrist_position_enhanced(frame, student_keypoints)
            
            # Compare with reference using normalized coordinates
            comparison = compare_poses_with_reference(student_keypoints, reference_data['keypoints'], relaxed_thresholds)
            
            if comparison:
                # Draw boxes based on current frame match status only
                for keypoint_name, data in student_keypoints.items():
                    if data is not None:
                        x, y = data[0], data[1]
                        
                        match_info = comparison['keypoint_matches'].get(keypoint_name, {})
                        
                        # GREEN for matched, RED for deviations (real-time only)
                        if match_info.get('matched', False):
                            color = (0, 255, 0)  # GREEN - CORRECT
                            thickness = 3
                        else:
                            color = (0, 0, 255)  # RED - DEVIATION (current frame only)
                            thickness = 4
                        
                        # Special handling for chest - use shoulder width
                        if keypoint_name == 'chest' and len(data) > 3:
                            shoulder_width = data[3]
                            half_width = shoulder_width // 2
                            half_height = 15
                            
                            # Draw wider rectangle spanning chest
                            cv2.rectangle(frame, 
                                         (x - half_width, y - half_height), 
                                         (x + half_width, y + half_height), 
                                         color, thickness) 
                            # Add label with deviation info
                            label = 'chest'
                            
                            # Add deviation description for mismatches
                            if not match_info.get('matched', False):
                                deviation = match_info.get('deviation_type', 'deviation')
                                label += f" - {deviation}"
                            
                            # Position label above the box
                            label_y = y - half_height - 8
                            
                            # Add semi-transparent background for text
                            (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.3, 1)
                            
                            overlay = frame.copy()
                            cv2.rectangle(overlay, 
                                         (x - half_width - 2, label_y - text_height - 3),
                                         (x - half_width + text_width + 2, label_y + 3),
                                         color, -1)
                            cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)
                            
                            cv2.putText(frame, label, 
                                       (x - half_width, label_y),
                                       cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA)
                        else:
                            # Standard box for other body parts
                            half_size = body_part_sizes.get(keypoint_name, 20) // 2
                            
                            # Draw rectangle
                            cv2.rectangle(frame, 
                                         (x - half_size, y - half_size), 
                                         (x + half_size, y + half_size), 
                                         color, thickness) 
                            # Add label with deviation info
                            label = keypoint_name.replace('_', ' ')
                            
                            # Add wrist position
                            if keypoint_name == 'left_hand':
                                label += f" [{student_wrist_pos['left_wrist']}]"
                            elif keypoint_name == 'right_hand':
                                label += f" [{student_wrist_pos['right_wrist']}]"
                            
                            # Add deviation description for mismatches
                            if not match_info.get('matched', False):
                                deviation = match_info.get('deviation_type', 'deviation')
                                label += f" - {deviation}"
                            
                            # Position label above the box
                            label_y = y - half_size - 8
                            
                            # Add semi-transparent background for text
                            (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.3, 1)
                            
                            overlay = frame.copy()
                            cv2.rectangle(overlay, 
                                         (x - half_size - 2, label_y - text_height - 3),
                                         (x - half_size + text_width + 2, label_y + 3),
                                         color, -1)
                            cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)
                            
                            cv2.putText(frame, label, 
                                       (x - half_size, label_y),
                                       cv2.FONT_HERSHEY_SIMPLEX, 0.3, (255, 255, 255), 1, cv2.LINE_AA)
                
                # Add info overlay
                match_pct = comparison['match_percentage']
                matched = comparison['matched_count']
                total = comparison['total_count']
                
                # Update cumulative average
                cumulative_match_sum += match_pct
                cumulative_avg = cumulative_match_sum / len(frame_results) if frame_results else match_pct
                
                # Create info panel at top
                overlay = frame.copy()
                cv2.rectangle(overlay, (0, 0), (width, 120), (0, 0, 0), -1)
                cv2.addWeighted(overlay, 0.6, frame, 0.4, 0, frame)
                
                # Color code the cumulative average percentage
                if cumulative_avg >= 70:
                    perf_color = (0, 255, 0)  # GREEN
                elif cumulative_avg >= 50:
                    perf_color = (0, 255, 255)  # YELLOW
                else:
                    perf_color = (0, 0, 255)  # RED
                
                cv2.putText(frame, f"Avg Score: {cumulative_avg:.1f}%", 
                           (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, perf_color, 2, cv2.LINE_AA)
                
                # Add wrist position info
                wrist_info = f"Wrists: L-{student_wrist_pos['left_wrist']} | R-{student_wrist_pos['right_wrist']}"
                cv2.putText(frame, wrist_info, 
                           (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                
                cv2.putText(frame, f"Frame: {frame_count}/{total_frames}", 
                           (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1, cv2.LINE_AA)
                
                frame_results.append({
                    'frame': frame_count,
                    'match_percentage': match_pct,
                    'comparison': comparison,
                    'wrist_positions': student_wrist_pos
                })
        
        # Write frame to output
        out.write(frame)
        
        if frame_count % 30 == 0:
            print(f"  Processed {frame_count}/{total_frames} frames...")
    
    cap.release()
    out.release()
    
    print(f"\n✓ Output video saved to: {output_path}")
    print(f"✓ Processed {frame_count} frames")
    
    return frame_results


# Process videos.mp4 → analyzed.mp4
print("\n" + "="*60)
print("ANALYZING VIDEO: videos.mp4")
print("="*60)
video_results = process_and_analyze_student_video(videos['input'], reference_data, model, output_video, relaxed_thresholds=False)

print("\n" + "="*60)
print("VIDEO PROCESSING COMPLETE")
print("="*60)


STEP 1: Creating Perfect Reference Video (NIKA3)

CREATING PERFECT REFERENCE VIDEO (NIKA3)

Video properties:
  Resolution: 1920x1080
  FPS: 30.0
  Total frames: 221

Processing frames...
  Processed 30/221 frames...
  Processed 30/221 frames...
  Processed 60/221 frames...
  Processed 60/221 frames...
  Processed 90/221 frames...
  Processed 90/221 frames...
  Processed 120/221 frames...
  Processed 120/221 frames...
  Processed 150/221 frames...
  Processed 150/221 frames...
  Processed 180/221 frames...
  Processed 180/221 frames...
  Processed 210/221 frames...
  Processed 210/221 frames...

✓ Perfect reference video saved to: c:\Users\sapok\Documents\GitHub\MuayML\nika3_perfect_reference.mp4
✓ Processed 221 frames

STEP 2: Analyzing NikaData against Nika3 (Perfect Reference)

PROCESSING STUDENT VIDEO

Video properties:
  Resolution: 1920x1080
  FPS: 30.0
  Total frames: 435

🎯 Using NORMALIZED COMPARISON (scale-independent)
🎯 Comparing with NIKA3 (Perfect Reference)

Processing f

In [ ]:
# Generate comprehensive analysis and grading
def generate_analysis_report(student_results, output_video_name):
    """Generate comprehensive analysis report."""
    print("\n" + "="*60)
    print("GENERATING ANALYSIS REPORT")
    print("="*60)
    
    if not student_results:
        print("⚠️ No results to analyze.")
        return
    
    # Calculate overall statistics
    total_frames_analyzed = len(student_results)
    
    if total_frames_analyzed > 0:
        avg_match_percentage = np.mean([r['match_percentage'] for r in student_results])
        
        # Calculate grade (1-100) based on match with reference
        grade = int(avg_match_percentage)
        
        # Generate detailed report
        print("\n📊 PERFORMANCE ANALYSIS")
        print("="*60)
        print(f"Reference: NIKA3 (Perfect)")
        print(f"Total Frames Analyzed: {total_frames_analyzed}")
        print(f"Average Match Percentage: {avg_match_percentage:.2f}%")
        print(f"\n⭐ OVERALL SCORE: {grade}/100")
        
        # Performance assessment
        print("\n💬 ASSESSMENT")
        print("-" * 60)
        if grade >= 90:
            assessment = "EXCELLENT! Form closely matches the perfect reference."
            emoji = "🏆"
        elif grade >= 75:
            assessment = "GOOD! Form is solid with some minor deviations."
            emoji = "✅"
        elif grade >= 60:
            assessment = "FAIR. Form shows promise but needs improvement."
            emoji = "⚠️"
        elif grade >= 40:
            assessment = "NEEDS WORK. Several areas need attention."
            emoji = "❌"
        else:
            assessment = "BEGINNER. Focus on mastering the basic form."
            emoji = "📚"
        
        print(f"{emoji} {assessment}")
        
        # Keypoint-specific analysis with deviation types
        print("\n🎯 DETAILED BODY PART ANALYSIS")
        print("="*60)
        
        # Aggregate keypoint match rates and deviation types
        keypoint_stats = {kp: {
            'matched': 0, 
            'total': 0, 
            'avg_distance': [],
            'deviation_types': []
        } for kp in KEYPOINT_MAPPING.keys()}
        
        for result in student_results:
            for kp_name, match_info in result['comparison']['keypoint_matches'].items():
                keypoint_stats[kp_name]['total'] += 1
                if match_info['matched']:
                    keypoint_stats[kp_name]['matched'] += 1
                if match_info['distance'] != float('inf'):
                    keypoint_stats[kp_name]['avg_distance'].append(match_info['distance'])
                if not match_info['matched'] and match_info.get('deviation_type'):
                    keypoint_stats[kp_name]['deviation_types'].append(match_info['deviation_type'])
        
        print(f"\n{'Body Part':<18} {'Score':<8} {'Status':<12} {'Common Issues'}")
        print("-" * 80)
        
        poor_parts = []
        
        for kp_name, stats in keypoint_stats.items():
            if stats['total'] > 0:
                match_rate = (stats['matched'] / stats['total']) * 100
                
                if match_rate >= 80:
                    status = "✓ EXCELLENT"
                elif match_rate >= 60:
                    status = "✓ GOOD"
                elif match_rate >= 40:
                    status = "⚠ NEEDS WORK"
                    poor_parts.append((kp_name, match_rate, stats))
                else:
                    status = "✗ POOR"
                    poor_parts.append((kp_name, match_rate, stats))
                
                # Get most common deviation
                common_issue = "None"
                if stats['deviation_types']:
                    from collections import Counter
                    deviation_counts = Counter(stats['deviation_types'])
                    most_common = deviation_counts.most_common(1)
                    if most_common:
                        common_issue = most_common[0][0]
                
                print(f"{kp_name.replace('_', ' ').title():<18} {match_rate:>5.1f}%  {status:<12} {common_issue}")
        
        # Biggest differences summary
        print("\n🔍 BIGGEST DIFFERENCES FROM PERFECT FORM")
        print("="*60)
        
        if poor_parts:
            poor_parts.sort(key=lambda x: x[1])
            
            print("\nTop areas needing improvement:\n")
            for i, (kp_name, match_rate, stats) in enumerate(poor_parts[:5], 1):
                body_part = kp_name.replace('_', ' ').title()
                
                if stats['deviation_types']:
                    from collections import Counter
                    deviation_counts = Counter(stats['deviation_types'])
                    top_issues = deviation_counts.most_common(2)
                    
                    issue_text = " and ".join([f"{issue}" for issue, count in top_issues])
                    print(f"  {i}. {body_part}: {issue_text} ({100-match_rate:.0f}% of frames)")
                else:
                    print(f"  {i}. {body_part}: Positioning needs correction ({100-match_rate:.0f}% of frames)")
        else:
            print("✓ All body parts show excellent form!")
        
        # Specific recommendations
        print("\n💡 SPECIFIC RECOMMENDATIONS")
        print("="*60)
        
        recommendations = []
        
        for kp_name, stats in keypoint_stats.items():
            if stats['total'] > 0:
                match_rate = (stats['matched'] / stats['total']) * 100
                
                if match_rate < 60 and stats['deviation_types']:
                    from collections import Counter
                    deviation_counts = Counter(stats['deviation_types'])
                    most_common = deviation_counts.most_common(1)[0]
                    
                    body_part = kp_name.replace('_', ' ').title()
                    
                    if 'too low' in most_common[0]:
                        recommendations.append(f"Raise your {body_part} higher")
                    elif 'too high' in most_common[0]:
                        recommendations.append(f"Lower your {body_part}")
                    elif 'too far left' in most_common[0]:
                        recommendations.append(f"Move your {body_part} more to the right")
                    elif 'too far right' in most_common[0]:
                        recommendations.append(f"Move your {body_part} more to the left")
                    elif 'major deviation' in most_common[0]:
                        recommendations.append(f"Focus on {body_part} positioning - significant adjustment needed")
        
        if recommendations:
            for i, rec in enumerate(recommendations[:5], 1):
                print(f"  {i}. {rec}")
        else:
            print("  ✓ Great job! Keep maintaining your current form.")
        
        # Wrist position analysis
        print("\n🤲 WRIST POSITION SUMMARY")
        print("-" * 60)
        
        left_wrist_positions = {'UP': 0, 'DOWN': 0, 'SIDEWAYS': 0, 'NEUTRAL': 0, 'UNKNOWN': 0}
        right_wrist_positions = {'UP': 0, 'DOWN': 0, 'SIDEWAYS': 0, 'NEUTRAL': 0, 'UNKNOWN': 0}
        
        for result in student_results:
            if 'wrist_positions' in result:
                wrist_pos = result['wrist_positions']
                left_wrist_positions[wrist_pos['left_wrist']] += 1
                right_wrist_positions[wrist_pos['right_wrist']] += 1
        
        print("\nLeft Wrist:")
        for pos, count in sorted(left_wrist_positions.items(), key=lambda x: x[1], reverse=True):
            if count > 0:
                percentage = (count / total_frames_analyzed) * 100
                print(f"  {pos:<10}: {percentage:>5.1f}%")
        
        print("\nRight Wrist:")
        for pos, count in sorted(right_wrist_positions.items(), key=lambda x: x[1], reverse=True):
            if count > 0:
                percentage = (count / total_frames_analyzed) * 100
                print(f"  {pos:<10}: {percentage:>5.1f}%")
        
        print("\n" + "="*60)
        print("📹 OUTPUT VIDEO")
        print("-" * 60)
        print(f"  ✓ Analysis: {output_video_name}")
        print("="*60)
        
        print(f"\n🎯 FINAL SCORE: {grade}/100")
        print(f"   {emoji} {assessment}")


# Generate report
generate_analysis_report(video_results, output_video.name)

print("\n" + "="*60)
print("✓ ANALYSIS COMPLETE")
print(f"✓ Input: videos.mp4")
print(f"✓ Output: analyzed.mp4")
print("="*60)


GENERATING ANALYSIS REPORT FOR NIKADATA

📊 PERFORMANCE ANALYSIS
Student: NIKADATA
Reference: NIKA3 (Perfect)
Total Frames Analyzed: 435
Average Match Percentage: 89.85%

⭐ OVERALL SCORE: 89/100

💬 ASSESSMENT
------------------------------------------------------------
✅ GOOD! Form is solid with some minor deviations.

🎯 DETAILED BODY PART ANALYSIS

Body Part          Score    Status       Common Issues
--------------------------------------------------------------------------------
Head               100.0%  ✓ EXCELLENT  None
Left Elbow         100.0%  ✓ EXCELLENT  None
Right Elbow         96.8%  ✓ EXCELLENT  too low, moderate deviation
Left Hand           65.5%  ✓ GOOD       too low, major deviation
Right Hand          59.5%  ⚠ NEEDS WORK too low, major deviation
Left Hip           100.0%  ✓ EXCELLENT  None
Right Hip          100.0%  ✓ EXCELLENT  None
Chest              100.0%  ✓ EXCELLENT  None
Left Knee          100.0%  ✓ EXCELLENT  None
Right Knee          99.1%  ✓ EXCELLENT  too 